In [1]:
from scipy.stats import entropy
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
from multiprocessing import Pool
import random

In [2]:
train = pd.read_csv("data\motionStrike_TVcodes_data.tsv\motionStrike_TVcodes_data.tsv", delimiter='\t' )
train.head()

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\aarya\AppData\Local\Temp\ipykernel_8000\3253002523.py:1: SyntaxWarning: invalid escape sequence '\m'
  train = pd.read_csv("data\motionStrike_TVcodes_data.tsv\motionStrike_TVcodes_data.tsv", delimiter='\t' )


,CaseReferenceNumber,MotionID,CaseLocation,CaseMajorCode,CaseMinorCode,CaseTrialListType,CaseMarkingCode,CaseDispositionJudgeJurisNo,CaseDispositionDocketLegendCode,MotionResultCode,...,MotionJurisNumber,MotionDocumentPriorityCode,CaseAttorneyJuris,CaseAttorneyType,MotionTimeDuration,MotionDocumentTypeName,SelfRepBeforeMotionFileCount,SelfRepBeforeMotionDecidedCount,SelfRepBeforeMotionFileBool,SelfRepBeforeMotionDecidedBool
0,207,1751,FBT,V,1,9,16.0,416586.0,JDGDACT,GR,...,418027.0,30,418027.0,J,78.0,MOTION TO STRIKE,1,1,True,True
1,1472,10915,FBT,V,1,9,16.0,NaN,WDACT,DN,...,416586.0,30,416586.0,R,22.0,MOTION TO STRIKE,0,0,False,False
2,1494,11050,NNI,V,1,9,16.0,NaN,WDACT,GR,...,422392.0,30,422392.0,J,285.0,MOTION TO STRIKE,0,0,False,False
3,371,2708,LLI,T,90,-1,NaN,418028.0,SJD,OR,...,418028.0,30,418028.0,R,120.0,MOTION TO STRIKE,0,0,False,False
4,1422,10373,HHD,T,2,-1,NaN,422394.0,SJ,GR,...,81181.0,30,81181.0,R,153.0,MOTION TO STRIKE,0,0,False,False


In [3]:
specialtab = train[["CaseAttorneyJuris", "CaseMajorCode"]]
specialtab.head()

,CaseAttorneyJuris,CaseMajorCode
0,418027.0,V
1,416586.0,V
2,422392.0,V
3,418028.0,T
4,81181.0,T


In [4]:
def calculate_attorney_specialization(specialtab):
    # Make a copy of the input dataframe to avoid the warning
    df = specialtab.copy()
    
    # Group by attorney and count case types
    case_counts = pd.crosstab(df['CaseAttorneyJuris'], 
                             df['CaseMajorCode'])
    
    # Calculate entropy with Dirichlet smoothing for each attorney
    attorney_entropy = {}
    for attorney in case_counts.index:
        counts = case_counts.loc[attorney]
        
        # Apply Dirichlet smoothing (adding 1 to each count)
        smoothed_counts = counts + 1
        # Calculate proportions
        proportions = smoothed_counts / smoothed_counts.sum()
        # Calculate entropy
        attorney_entropy[attorney] = entropy(proportions, base=2)
    
    # Add entropy back to dataframe
    df['AttorneySpecialization'] = df['CaseAttorneyJuris'].map(attorney_entropy)
    
    return df
print("function passed")

function passed


In [5]:
# Use the function
specialentropy = calculate_attorney_specialization(specialtab)

# Look at entropy scores
print("\nEntropy scores:")
print(specialentropy.groupby('CaseAttorneyJuris')['AttorneySpecialization'].head())

print("\nMost specialized attorneys (lowest entropy):")
print(specialentropy.groupby('CaseAttorneyJuris')
      .agg({'AttorneySpecialization': 'first', 'CaseMajorCode': lambda x: x.value_counts().to_dict()})
      .sort_values('AttorneySpecialization')
      .head())


Entropy scores:
0        0.999273
1        0.992774
2        0.999571
3        0.902393
4        0.977001
           ...   
13730    0.954434
13752    0.985228
13760    0.954434
13768    0.985228
13776    0.985228
Name: AttorneySpecialization, Length: 1276, dtype: float64

Most specialized attorneys (lowest entropy):
                   AttorneySpecialization      CaseMajorCode
CaseAttorneyJuris                                           
407897.0                         0.286397          {'T': 18}
412189.0                         0.391244          {'T': 11}
435703.0                         0.413817          {'T': 10}
80808.0                          0.413817          {'T': 10}
81240.0                          0.422001  {'T': 31, 'V': 2}


In [6]:
legal = train[["CaseLocation","CaseMajorCode","MotionJurisNumber","MotionResultCode"]]
legal.head()


,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode
0,FBT,V,418027.0,GR
1,FBT,V,416586.0,DN
2,NNI,V,422392.0,GR
3,LLI,T,418028.0,OR
4,HHD,T,81181.0,GR


In [7]:
filtered = legal[legal["MotionResultCode"].isin(["GR","DN"])]
filtered.head()

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode
0,FBT,V,418027.0,GR
1,FBT,V,416586.0,DN
2,NNI,V,422392.0,GR
4,HHD,T,81181.0,GR
6,MMX,T,403965.0,DN


In [8]:
#apply one hot to MotionResultCode
#make this a function later
motion_GRDR = pd.get_dummies(filtered["MotionResultCode"],  prefix='MotionResultCode')
motion_GRDR = motion_GRDR.astype(int)
motion_GRDR.head()

,MotionResultCode_DN,MotionResultCode_GR
0,0,1
1,1,0
2,0,1
4,0,1
6,1,0


In [9]:
#apply onehot to caselocation
case_location =  pd.get_dummies(train["CaseLocation"], prefix = "CaseLocation")
case_location = case_location.astype(int)
case_location.head()

,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,CaseLocation_FST,CaseLocation_HHB,CaseLocation_HHD,CaseLocation_KNL,CaseLocation_KNO,CaseLocation_LLI,CaseLocation_MMX,CaseLocation_NNH,CaseLocation_NNI,CaseLocation_TTD,CaseLocation_UWY,CaseLocation_WWM
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [10]:
#apply to case attorney type
attorney_type =  pd.get_dummies(train["CaseAttorneyType"], prefix = "CaseAttorneyType")
attorney_type = attorney_type.astype(int)
attorney_type.head()


,CaseAttorneyType_A,CaseAttorneyType_F,CaseAttorneyType_G,CaseAttorneyType_J,CaseAttorneyType_K,CaseAttorneyType_M,CaseAttorneyType_Q,CaseAttorneyType_R,CaseAttorneyType_U,CaseAttorneyType_X
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0


In [11]:
#apply to CaseDispositionDocketLegendCode
disp_docket =  pd.get_dummies(train["CaseDispositionDocketLegendCode"], prefix = "CaseDispositionDocketLegendCode")
disp_docket = disp_docket.astype(int)
disp_docket.head()

,CaseDispositionDocketLegendCode_ARBJDG,CaseDispositionDocketLegendCode_FFJDG,CaseDispositionDocketLegendCode_JDGACT,CaseDispositionDocketLegendCode_JDGACTD,CaseDispositionDocketLegendCode_JDGACTP,CaseDispositionDocketLegendCode_JDGDACT,CaseDispositionDocketLegendCode_JDGHD,CaseDispositionDocketLegendCode_JDGMENT,CaseDispositionDocketLegendCode_JDGNST,CaseDispositionDocketLegendCode_JDGRPT,...,CaseDispositionDocketLegendCode_SATJFBS,CaseDispositionDocketLegendCode_SJ,CaseDispositionDocketLegendCode_SJD,CaseDispositionDocketLegendCode_SJP,CaseDispositionDocketLegendCode_TOUSDCT,CaseDispositionDocketLegendCode_TTFSTHS,CaseDispositionDocketLegendCode_WDACT,CaseDispositionDocketLegendCode_WDCWC,CaseDispositionDocketLegendCode_WDRAPJ,CaseDispositionDocketLegendCode_WOARD
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
#Beta Bernouli for CaseMajorCode
#Tort ==1 Vehicular == 0 
#turn into function

#DO NOT RUN THIS CELL AFTER RAN ONCE WILL FILTER ON TOP OF WHAT IS ALREADY FILTERED 

#we want two columns do the one hot approach for now 
"""
filtered['CaseMajorCode'] = filtered['CaseMajorCode'].apply(lambda x: 1 if x == 'T' else 0)=
filtered
"""
case_major = pd.get_dummies(train["CaseMajorCode"], prefix = "CaseMajorCode")
case_major = case_major.astype(int)
case_major.head()

,CaseMajorCode_T,CaseMajorCode_V
0,0,1
1,0,1
2,0,1
3,1,0
4,1,0


In [13]:

def clean_dataset(dataset):
    # Drop columns with any NaN or Null values
    dataset = dataset.dropna(axis=1)
    #drops with blank values
    dataset = dataset.loc[:, ~(dataset == '').any()] 
    return dataset

    

In [15]:
"""
CREATE the FULL SUBSET AND MINIMAL DATA GROUPS

FULL
'CaseAttorneyJuris', 'CaseLocation_AAN',
                           'CaseLocation_DBD', 'CaseLocation_FBT', 'CaseLocation_FST',
                           'CaseLocation_HHB', 'CaseLocation_HHD', 'CaseLocation_KNL',
                           'CaseLocation_KNO', 'CaseLocation_LLI', 'CaseLocation_MMX',
                           'CaseLocation_NNH', 'CaseLocation_NNI', 'CaseLocation_TTD',
                           'CaseLocation_UWY', 'CaseLocation_WWM', 'CaseLocation_nan',
                           'CaseAttorneyType_A', 'CaseAttorneyType_G',
                           'CaseAttorneyType_J', 'CaseAttorneyType_K', 'CaseAttorneyType_M',
                           'CaseAttorneyType_Q', 'CaseAttorneyType_R', 'CaseAttorneyType_U',
                           'CaseAttorneyType_X', 'CaseAttorneyType_nan',
                           'CaseDispositionDocketLegendCode_ARBJDG',
                           'CaseDispositionDocketLegendCode_FFJDG',
                           'CaseDispositionDocketLegendCode_JDGACT',
                           'CaseDispositionDocketLegendCode_JDGACTD',
                           'CaseDispositionDocketLegendCode_JDGACTP',
                           'CaseDispositionDocketLegendCode_JDGDACT',
                           'CaseDispositionDocketLegendCode_JDGHD',
                           'CaseDispositionDocketLegendCode_JDGMENT',
                           'CaseDispositionDocketLegendCode_JDGNST',
                           'CaseDispositionDocketLegendCode_JDGRPT',
                           'CaseDispositionDocketLegendCode_JDGSTA',
                           'CaseDispositionDocketLegendCode_JDGSTAP',
                           'CaseDispositionDocketLegendCode_JDGSTD',
                           'CaseDispositionDocketLegendCode_JDGSTP',
                           'CaseDispositionDocketLegendCode_JDGSTPP',
                           'CaseDispositionDocketLegendCode_JDGTVT',
                           'CaseDispositionDocketLegendCode_JDGTVTD',
                           'CaseDispositionDocketLegendCode_JDGTVTP',
                           'CaseDispositionDocketLegendCode_JDJVTCT',
                           'CaseDispositionDocketLegendCode_JGNSA',
                           'CaseDispositionDocketLegendCode_JGVMAPC',
                           'CaseDispositionDocketLegendCode_JOD',
                           'CaseDispositionDocketLegendCode_JODD',
                           'CaseDispositionDocketLegendCode_JWT',
                           'CaseDispositionDocketLegendCode_JWTD',
                           'CaseDispositionDocketLegendCode_JWTP',
                           'CaseDispositionDocketLegendCode_SATJFBS',
                           'CaseDispositionDocketLegendCode_SJ',
                           'CaseDispositionDocketLegendCode_SJD',
                           'CaseDispositionDocketLegendCode_SJP',
                           'CaseDispositionDocketLegendCode_TOUSDCT',
                           'CaseDispositionDocketLegendCode_TTFSTHS',
                           'CaseDispositionDocketLegendCode_WDACT',
                           'CaseDispositionDocketLegendCode_WDCWC',
                           'CaseDispositionDocketLegendCode_WDRAPJ',
                           'CaseDispositionDocketLegendCode_WOARD',
                           'CaseDispositionDocketLegendCode_nan',
                           'CaseDispositionJudgeJurisNo',
                           'CaseMajorCode_T', 'CaseMajorCode_V', 'CaseMajorCode_nan',
                           'CaseMarkingCode', 'CaseMinorCode', 'CaseReferenceNumber',
                           'CaseTrialListType', 'MotionJurisNumber',
                           'MotionTimeDuration', 'SelfRepBeforeMotionDecidedBool',
                           'SelfRepBeforeMotionDecidedCount', 'SelfRepBeforeMotionFileBool',
                           'SelfRepBeforeMotionFileCount',
                           "MotionResultCode_GR"]



"""


'\nCREATE the FULL SUBSET AND MINIMAL DATA GROUPS\n\nFULL\n\'CaseAttorneyJuris\', \'CaseLocation_AAN\',\n                           \'CaseLocation_DBD\', \'CaseLocation_FBT\', \'CaseLocation_FST\',\n                           \'CaseLocation_HHB\', \'CaseLocation_HHD\', \'CaseLocation_KNL\',\n                           \'CaseLocation_KNO\', \'CaseLocation_LLI\', \'CaseLocation_MMX\',\n                           \'CaseLocation_NNH\', \'CaseLocation_NNI\', \'CaseLocation_TTD\',\n                           \'CaseLocation_UWY\', \'CaseLocation_WWM\', \'CaseLocation_nan\',\n                           \'CaseAttorneyType_A\', \'CaseAttorneyType_G\',\n                           \'CaseAttorneyType_J\', \'CaseAttorneyType_K\', \'CaseAttorneyType_M\',\n                           \'CaseAttorneyType_Q\', \'CaseAttorneyType_R\', \'CaseAttorneyType_U\',\n                           \'CaseAttorneyType_X\', \'CaseAttorneyType_nan\',\n                           \'CaseDispositionDocketLegendCode_ARBJDG\',\

In [16]:
#combine all of the one_hot categories into the train data frame
#special entropy
#filtered
#all into train 

#next: 
#pd.merge to comine then run the new set on the dataset?

#next:
#run train_test_split
#our output is Motion Result Code _GR
#so that would be our y value 

#next: 
#create the classifiers and run 
# within the classifiers run GridSearch to identify the best hyperparameters for each classifier



#finished

In [14]:
#combining alll into final data frame 
full_data = specialentropy["AttorneySpecialization"]


combined_df = pd.concat([full_data, filtered, motion_GRDR, case_location, attorney_type, disp_docket, case_major], axis = 1)
combined_df = combined_df.drop("MotionResultCode", axis = 1)


#now combine with the train value
"""
CaseAttorneyJuris, CaseMinorCode, CaseReferenceNumber, CaseTrialListType, MotionJurisNumber, MotionTimeDuration, 
SelfRepBeforeMotionDecidedBool, SelfRepBeforeMotionDecidedCount, SelfRepBeforeMotionFileBool, SelfRepBeforeMotionFileCount

"""
combined_df

,AttorneySpecialization,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode_DN,MotionResultCode_GR,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,CaseLocation_FST,...,CaseDispositionDocketLegendCode_SJD,CaseDispositionDocketLegendCode_SJP,CaseDispositionDocketLegendCode_TOUSDCT,CaseDispositionDocketLegendCode_TTFSTHS,CaseDispositionDocketLegendCode_WDACT,CaseDispositionDocketLegendCode_WDCWC,CaseDispositionDocketLegendCode_WDRAPJ,CaseDispositionDocketLegendCode_WOARD,CaseMajorCode_T,CaseMajorCode_V
0,0.999273,FBT,V,418027.0,0.0,1.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0.992774,FBT,V,416586.0,1.0,0.0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,0.999571,NNI,V,422392.0,0.0,1.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.902393,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0.977001,HHD,T,81181.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798,0.980867,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13799,0.884115,HHB,T,431662.0,1.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13800,0.991076,TTD,T,436946.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13801,0.989588,FBT,T,438581.0,0.0,1.0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
initial = train[["CaseReferenceNumber","CaseAttorneyJuris", "CaseMinorCode", "CaseReferenceNumber", "CaseTrialListType", "MotionJurisNumber", "MotionTimeDuration", "SelfRepBeforeMotionDecidedBool", "SelfRepBeforeMotionDecidedCount", "SelfRepBeforeMotionFileBool", "SelfRepBeforeMotionFileCount"]]
full_set = pd.concat([combined_df, initial], axis = 1)

full_set = full_set.drop("CaseMajorCode", axis =1)
full_set = full_set.drop("CaseLocation", axis =1)
full_set.head()



,AttorneySpecialization,MotionJurisNumber,MotionResultCode_DN,MotionResultCode_GR,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,CaseLocation_FST,CaseLocation_HHB,CaseLocation_HHD,...,CaseAttorneyJuris,CaseMinorCode,CaseReferenceNumber,CaseTrialListType,MotionJurisNumber,MotionTimeDuration,SelfRepBeforeMotionDecidedBool,SelfRepBeforeMotionDecidedCount,SelfRepBeforeMotionFileBool,SelfRepBeforeMotionFileCount
0,0.999273,418027.0,0.0,1.0,0,0,1,0,0,0,...,418027.0,1,207,9,418027.0,78.0,True,1,True,1
1,0.992774,416586.0,1.0,0.0,0,0,1,0,0,0,...,416586.0,1,1472,9,416586.0,22.0,False,0,False,0
2,0.999571,422392.0,0.0,1.0,0,0,0,0,0,0,...,422392.0,1,1494,9,422392.0,285.0,False,0,False,0
3,0.902393,NaN,NaN,NaN,0,0,0,0,0,0,...,418028.0,90,371,-1,418028.0,120.0,False,0,False,0
4,0.977001,81181.0,0.0,1.0,0,0,0,0,0,1,...,81181.0,2,1422,-1,81181.0,153.0,False,0,False,0


In [16]:
#subset
"""
SUBSET
['MotionJurisNumber', 'CaseMajorCode_T', 'CaseMajorCode_V',
                             'MotionTimeDuration', 'CaseLocation_AAN',
                             'CaseLocation_DBD', 'CaseLocation_FBT', 'CaseLocation_FST',
                             'CaseLocation_HHB', 'CaseLocation_HHD', 'CaseLocation_KNL',
                             'CaseLocation_KNO', 'CaseLocation_LLI', 'CaseLocation_MMX',
                             'CaseLocation_NNH', 'CaseLocation_NNI', 'CaseLocation_TTD',
                             'CaseLocation_UWY', 'CaseLocation_WWM', "MotionResultCode_GR,
                             'CaseLocation_NNH', 'CaseLocation_NNI', 'CaseLocation_TTD',
                             'CaseLocation_UWY', 'CaseLocation_WWM', "MotionResultCode_GR"]
"""

subset = full_set[["CaseReferenceNumber", "AttorneySpecialization", "MotionJurisNumber",
                   "CaseMajorCode_T", "CaseMajorCode_V", "MotionTimeDuration",
                   "CaseLocation_AAN", "CaseLocation_DBD", "CaseLocation_FBT",
                   "CaseLocation_FST", "CaseLocation_HHB", "CaseLocation_HHD",
                   "CaseLocation_KNL", "CaseLocation_KNO", "CaseLocation_LLI",
                   "CaseLocation_MMX", "CaseLocation_NNH", "CaseLocation_NNI",
                   "CaseLocation_TTD", "CaseLocation_UWY", "CaseLocation_WWM",
                   "MotionResultCode_GR"]]

def drop_duplicate_columns(df):
    """
    Drops duplicate columns in a pandas DataFrame, keeping only the first occurrence.

    Args:
        df: The input pandas DataFrame.

    Returns:
        A new DataFrame with duplicate columns removed.
    """

    # Transpose the DataFrame, drop duplicate rows, and then transpose back
    df_deduped = df.T.drop_duplicates().T

    return df_deduped

subset = drop_duplicate_columns(subset)

subset

,CaseReferenceNumber,AttorneySpecialization,MotionJurisNumber,MotionJurisNumber,CaseMajorCode_T,CaseMajorCode_V,MotionTimeDuration,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,...,CaseLocation_KNL,CaseLocation_KNO,CaseLocation_LLI,CaseLocation_MMX,CaseLocation_NNH,CaseLocation_NNI,CaseLocation_TTD,CaseLocation_UWY,CaseLocation_WWM,MotionResultCode_GR
0,207.0,0.999273,418027.0,418027.0,0.0,1.0,78.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1472.0,0.992774,416586.0,416586.0,0.0,1.0,22.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1494.0,0.999571,422392.0,422392.0,0.0,1.0,285.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,371.0,0.902393,NaN,418028.0,1.0,0.0,120.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1422.0,0.977001,81181.0,81181.0,1.0,0.0,153.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798,4137836.0,0.980867,NaN,415596.0,1.0,0.0,47.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13799,4143717.0,0.884115,431662.0,431662.0,1.0,0.0,49.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13800,4111125.0,0.991076,436946.0,436946.0,1.0,0.0,47.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
13801,4026253.0,0.989588,438581.0,438581.0,1.0,0.0,47.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
#minimal
"""

MINIMAL

 ['CaseAttorneyJuris',
                              'MotionTimeDuration',
                              "MotionResultCode_GR"]

"""
minimal = full_set[["CaseReferenceNumber", "AttorneySpecialization", "CaseAttorneyJuris", "MotionTimeDuration", "MotionResultCode_GR"]]

minimal = drop_duplicate_columns(minimal)
minimal

,CaseReferenceNumber,AttorneySpecialization,CaseAttorneyJuris,MotionTimeDuration,MotionResultCode_GR
0,207.0,0.999273,418027.0,78.0,1.0
1,1472.0,0.992774,416586.0,22.0,0.0
2,1494.0,0.999571,422392.0,285.0,1.0
3,371.0,0.902393,418028.0,120.0,NaN
4,1422.0,0.977001,81181.0,153.0,1.0
...,...,...,...,...,...
13798,4137836.0,0.980867,415596.0,47.0,NaN
13799,4143717.0,0.884115,431662.0,49.0,0.0
13800,4111125.0,0.991076,436946.0,47.0,1.0
13801,4026253.0,0.989588,438581.0,47.0,1.0


In [18]:

# Save to csv
full_set.to_csv("Full_TAB", index = False)
subset.to_csv("Subset_TAB", index = False)
minimal.to_csv("Minimal_TAB", index= False)

In [20]:
"""

THIS CELL DON'T RUN CONVERSION INTEGRATED IN "Final_Tf_IDF_and_Word2vec.ipynb"

KEEP FOR POTENTIAL ERRORS AND METHOD CHANGES 

"""


#combine

#word embeddings 
#assign each word embeddings doc id to caserefnumber 

#first convert embeddings to dataframe 
def numpy_to_dataframe(npy_file_path, column_prefix="feature"):
    try:
        # 1. Load the .npy file
        numpy_array = np.load(npy_file_path)

        # 2. Check the shape of the NumPy array
        print(f"Shape of NumPy array: {numpy_array.shape}")

        # 3. Convert NumPy array to Pandas DataFrame
        num_cols = numpy_array.shape[1] if len(numpy_array.shape) > 1 else 1  # Get the number of columns
        if num_cols > 1:
            column_names = [f'{column_prefix}_{i}' for i in range(num_cols)]
            df = pd.DataFrame(numpy_array, columns=column_names)
        else:
             df = pd.DataFrame(numpy_array)


        return df
    except FileNotFoundError:
        print(f"Error: File not found at {npy_file_path}")
        return None
    except Exception as e:
      print(f"Error: an error occurred {e}")
      return None  

word_embedding_file = "final_embeddings.npy"
word_embeddings = numpy_to_dataframe(word_embedding_file, column_prefix="embed")

if word_embeddings is not None:
      print("\nDataFrame Head:")
      print(word_embeddings.head())
else:
    print("Conversion failed, please check input data")


#this code messes up and gets rid of the docid need to restore the docid
#this means we need to fix the old code in our colab notebook 


Shape of NumPy array: (8217, 100)

DataFrame Head:
    embed_0   embed_1   embed_2   embed_3   embed_4   embed_5   embed_6  \
0 -0.027148  0.137064 -0.063844 -0.049998  0.067057  0.185018  0.165134   
1 -0.019730  0.157603 -0.269141  0.561909 -0.182994  0.267730  0.310358   
2 -0.071350  0.048779 -0.014560  0.335733 -0.241535  0.302115  0.253838   
3 -0.027707  0.016602  0.029347  0.308740 -0.257334  0.224717  0.180193   
4 -0.133086  0.118625 -0.141091  0.155188 -0.137553  0.232561  0.235799   

    embed_7   embed_8   embed_9  ...  embed_90  embed_91  embed_92  embed_93  \
0 -0.046370 -0.010687  0.040046  ... -0.095136  0.177535 -0.066742 -0.090948   
1 -0.233656 -0.227080  0.086266  ... -0.409305  0.176552 -0.085438 -0.396587   
2 -0.308555 -0.238537 -0.036342  ... -0.303897  0.466195  0.086223 -0.181438   
3 -0.220868 -0.144622 -0.055463  ... -0.266537  0.344464  0.005979 -0.125546   
4 -0.160968 -0.104497 -0.029423  ... -0.162386  0.408240  0.144622 -0.226297   

   embed_94  embe

In [71]:
#word2vec combined

#load the translation table
translation_table = "data\judcaseid_docid_translationtable.tsv\judcaseid_docid_translationtable.tsv"
translation_df = pd.read_csv(translation_table, sep='\t', names=['docid', 'CaseReferenceNumber'], skiprows=1)


#load the Word2Vec embeddings 
word2vec_embeddings = 'combined_embeddings.csv'
word2vec_embeddings = pd.read_csv(word2vec_embeddings)



#merging 
translation_word2vec = pd.merge(word2vec_embeddings, translation_df, on='docid', how='left')
      
merged_subset = pd.merge(subset, translation_word2vec, on='CaseReferenceNumber', how='left')



<>:4: SyntaxWarning: invalid escape sequence '\j'
<>:4: SyntaxWarning: invalid escape sequence '\j'
C:\Users\aarya\AppData\Local\Temp\ipykernel_8888\2503952386.py:4: SyntaxWarning: invalid escape sequence '\j'
  translation_table = "data\judcaseid_docid_translationtable.tsv\judcaseid_docid_translationtable.tsv"


In [ ]:
merged_subset.head()

,CaseReferenceNumber,AttorneySpecialization,MotionJurisNumber,MotionJurisNumber,CaseMajorCode_T,CaseMajorCode_V,MotionTimeDuration,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,...,CaseLocation_MMX,CaseLocation_NNH,CaseLocation_NNI,CaseLocation_TTD,CaseLocation_UWY,CaseLocation_WWM,MotionResultCode_GR,Unnamed: 0,docid,embedding
0,207.0,0.999273,418027.0,418027.0,0.0,1.0,78.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
1,1472.0,0.992774,416586.0,416586.0,0.0,1.0,22.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2246.0,"[-1.5055830225745082, -0.3105193272289667, 0.6..."
2,1494.0,0.999571,422392.0,422392.0,0.0,1.0,285.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,15.0,2566.0,"[0.011885851566811875, -0.26892964942287595, 0..."
3,371.0,0.902393,NaN,418028.0,1.0,0.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,1422.0,0.977001,81181.0,81181.0,1.0,0.0,153.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,2032.0,"[-0.23641143218466626, 0.6719418527549225, -0...."


In [70]:
#doc2vec combined


#load the Word2Vec embeddings 
doc2vec_embeddings = 'doc2vec_embeddings.csv'
doc2vec_embeddings = pd.read_csv(doc2vec_embeddings)
doc2vec_embeddings = doc2vec_embeddings.drop(index=0)


#merge


translation_doc2vec = pd.merge(doc2vec_embeddings, translation_df, on='docid', how='left')
merged_doc = pd.merge(subset, translation_doc2vec, on='CaseReferenceNumber', how='left')
   
merged_doc.head()


,CaseReferenceNumber,AttorneySpecialization,MotionJurisNumber,MotionJurisNumber,CaseMajorCode_T,CaseMajorCode_V,MotionTimeDuration,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,...,CaseLocation_LLI,CaseLocation_MMX,CaseLocation_NNH,CaseLocation_NNI,CaseLocation_TTD,CaseLocation_UWY,CaseLocation_WWM,MotionResultCode_GR,docid,embeddings
0,207.0,0.999273,418027.0,418027.0,0.0,1.0,78.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
1,1472.0,0.992774,416586.0,416586.0,0.0,1.0,22.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2246.0,[ 0.08293848 -0.92715216 0.13057815 -1.316873...
2,1494.0,0.999571,422392.0,422392.0,0.0,1.0,285.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2566.0,[ 0.75967443 -1.1660833 0.10017195 -0.815574...
3,1422.0,0.977001,81181.0,81181.0,1.0,0.0,153.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2032.0,[-1.2798557 -0.54604745 -0.50894713 -0.781494...
4,4469.0,0.976310,403965.0,403965.0,1.0,0.0,79.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [72]:
#drop nan rows in all datasets
full_set = full_set.dropna(axis = 0, how = "any")
subset = subset.dropna(axis = 0, how = "any")
minimal = minimal.dropna(axis = 0, how = "any")
merged_subset = merged_subset.dropna(axis = 0, how = "any")
merged_doc = merged_doc.dropna(axis = 0, how = "any")

In [29]:
merged_subset.to_csv("scan.csv")

In [73]:
import ast



# Convert the 'embedding' column from string representation of lists to actual lists of floats

merged_subset['embedding'] = merged_subset['embedding'].apply(lambda x: [float(i) for i in ast.literal_eval(x)])




In [40]:
merged_doc.to_csv("doc2vec_test.csv")

In [90]:
import ast

# Fix the 'embeddings' column by removing invalid syntax and converting to a proper list
merged_doc['embeddings'] = (
    merged_doc['embeddings']
    .str.replace(r',,', ',', regex=True)  # Replace double commas with single comma
    .str.replace(r'\[\s*,', '[', regex=True)  # Remove commas after opening brackets
    .str.replace(r',\s*\]', ']', regex=True)  # Remove commas before closing brackets
    .apply(lambda x: [float(i) for i in ast.literal_eval(x)])  # Convert to list of floats
)




AttributeError: Can only use .str accessor with string values!

In [95]:
# Now the column should be cleaned and contain proper lists
merged_doc['embeddings'] = merged_doc['embeddings'].apply(
    lambda x: [float(i) for i in x] if isinstance(x, list) else x
)

# Verify the cleaned embeddings column
print(merged_doc['embeddings'].head())


1    [0.08293848, -0.92715216, 0.13057815, -1.31687...
2    [0.75967443, -1.1660833, 0.10017195, -0.815574...
3    [-1.2798557, -0.54604745, -0.50894713, -0.7814...
5    [0.2662329, -0.6008199, -0.37289077, -0.872666...
7    [0.02245815, -0.73424727, -0.97657, -0.3100437...
Name: embeddings, dtype: object


In [93]:
is_float = all(all(isinstance(i, float) for i in item) for item in merged_doc['embeddings'])
print("All elements within lists are floats:", is_float)

All elements within lists are floats: True


In [33]:
is_float = all(all(isinstance(i, float) for i in item) for item in merged_subset['embedding'])
print("All elements within lists are floats:", is_float)

All elements within lists are floats: True


In [100]:
merged_doc.head()


,CaseReferenceNumber,AttorneySpecialization,MotionJurisNumber,MotionJurisNumber,CaseMajorCode_T,CaseMajorCode_V,MotionTimeDuration,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,...,CaseLocation_LLI,CaseLocation_MMX,CaseLocation_NNH,CaseLocation_NNI,CaseLocation_TTD,CaseLocation_UWY,CaseLocation_WWM,MotionResultCode_GR,docid,embeddings
1,1472.0,0.992774,416586.0,416586.0,0.0,1.0,22.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2246.0,"[0.08293848, -0.92715216, 0.13057815, -1.31687..."
2,1494.0,0.999571,422392.0,422392.0,0.0,1.0,285.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2566.0,"[0.75967443, -1.1660833, 0.10017195, -0.815574..."
3,1422.0,0.977001,81181.0,81181.0,1.0,0.0,153.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2032.0,"[-1.2798557, -0.54604745, -0.50894713, -0.7814..."
5,2048.0,0.999125,420175.0,420175.0,0.0,1.0,60.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6104.0,"[0.2662329, -0.6008199, -0.37289077, -0.872666..."
7,3414.0,0.999821,419018.0,419018.0,0.0,1.0,91.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15408.0,"[0.02245815, -0.73424727, -0.97657, -0.3100437..."


In [61]:
merged_subset.dtypes
merged_subset.to_csv("scat.csv")

In [110]:
merged_doc.head()
merged_doc.to_csv("tests.csv")

In [101]:
import numpy as np

# Create a new DataFrame to hold the merged data
new_data_out = merged_subset.copy()

# Assume 'embedding' column contains lists of embeddings
# Get the number of dimensions of the embeddings
embedding_dim = len(merged_subset['embedding'].iloc[0])  # Assuming the first row is valid

# Create new columns in `new_data_out` for each embedding dimension, initially filled with NaN
for i in range(embedding_dim):
    new_data_out[f'embedding_{i}'] = np.nan

# Populate the new embedding columns
for index, row in merged_subset.iterrows():
    # Skip rows with invalid embeddings
    if isinstance(row['embedding'], list) and len(row['embedding']) == embedding_dim:
        for j in range(embedding_dim):
            new_data_out.at[index, f'embedding_{j}'] = row['embedding'][j]

# Drop the original 'embedding' column
new_data_out = new_data_out.drop(columns=['embedding'])

# Display the prepared data
print(new_data_out.head())




KeyError: 'embedding'

In [103]:

embedding_dim = len(merged_doc['embeddings'].iloc[0])

# Create new columns for each embedding dimension
for i in range(embedding_dim):
    merged_doc[f'embedding_{i}'] = merged_doc['embeddings'].apply(lambda x: x[i] if isinstance(x, list) else None)

# Drop the original 'embeddings' column
merged_doc = merged_doc.drop(columns=['embeddings'])



C:\Users\aarya\AppData\Local\Temp\ipykernel_8888\3808593941.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_doc[f'embedding_{i}'] = merged_doc['embeddings'].apply(lambda x: x[i] if isinstance(x, list) else None)
C:\Users\aarya\AppData\Local\Temp\ipykernel_8888\3808593941.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_doc[f'embedding_{i}'] = merged_doc['embeddings'].apply(lambda x: x[i] if isinstance(x, list) else None)
C:\Users\aarya\AppData\Local\Temp\ipykernel_8888\3808593941.py:5: PerformanceWarning: Da

In [104]:
merged_doc.head()

,CaseReferenceNumber,AttorneySpecialization,MotionJurisNumber,MotionJurisNumber,CaseMajorCode_T,CaseMajorCode_V,MotionTimeDuration,CaseLocation_AAN,CaseLocation_DBD,CaseLocation_FBT,...,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99
1,1472.0,0.992774,416586.0,416586.0,0.0,1.0,22.0,0.0,0.0,1.0,...,1.555897,-0.433980,0.368283,-0.347268,2.142146,0.955145,-1.297216,-1.173927,0.669186,-0.262536
2,1494.0,0.999571,422392.0,422392.0,0.0,1.0,285.0,0.0,0.0,0.0,...,0.681935,0.801277,-0.162410,-0.111082,1.915118,1.434827,-0.980419,-0.539615,0.208913,0.634646
3,1422.0,0.977001,81181.0,81181.0,1.0,0.0,153.0,0.0,0.0,0.0,...,0.510317,0.046707,-0.844210,-0.249279,3.150621,2.449819,-2.002077,-0.543965,0.333183,-1.097493
5,2048.0,0.999125,420175.0,420175.0,0.0,1.0,60.0,0.0,0.0,0.0,...,0.396147,0.793631,0.685799,-0.258846,2.209261,0.734330,-1.000987,0.143851,-0.232391,0.232837
7,3414.0,0.999821,419018.0,419018.0,0.0,1.0,91.0,0.0,0.0,1.0,...,1.034331,-1.031813,1.071153,-1.167057,2.226282,0.879569,-0.572301,-2.638766,0.227437,-0.715994


In [75]:
merged_subset = new_data_out

In [105]:
#train test split just for the tabular data 
import sklearn
from sklearn.model_selection import train_test_split


#on subset 
x = subset.drop("MotionResultCode_GR", axis = 1)
y = subset[["MotionResultCode_GR"]]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify = y)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")


#minimal

x_min = minimal.drop("MotionResultCode_GR", axis = 1)
y_min = minimal[["MotionResultCode_GR"]]

X_min_train, X_min_test, y_min_train, y_min_test = train_test_split(x_min, y_min, test_size=0.3, random_state=42, stratify = y_min)

print(f"Shape of X_train (minimal): {X_min_train.shape}")
print(f"Shape of y_train (minimal): {y_min_train.shape}")
print(f"Shape of X_test (minimal): {X_min_test.shape}")
print(f"Shape of y_test (minimal): {y_min_test.shape}")



#merged - word2vec

x_merge = merged_subset.drop("MotionResultCode_GR", axis = 1)
y_merge = merged_subset[["MotionResultCode_GR"]]

X_merge_train, X_merge_test, y_merge_train, y_merge_test = train_test_split(x_merge, y_merge, test_size=0.3, random_state=42, stratify = y_merge)

print(f"Shape of X_train (word2vec): {X_merge_train.shape}")
print(f"Shape of y_train (word2vec): {y_merge_train.shape}")
print(f"Shape of X_test (word2vec): {X_merge_test.shape}")
print(f"Shape of y_test (word2vec): {y_merge_test.shape}")


#merged - doc2vec
x_doc2vec = merged_doc.drop("MotionResultCode_GR", axis = 1)
y_doc2vec = merged_doc[["MotionResultCode_GR"]]

X_doc_train, X_doc_test, y_doc_train, y_doc_test = train_test_split(x_doc2vec, y_doc2vec, test_size=0.3, random_state=42, stratify = y_doc2vec)


print(f"Shape of X_train (doc2vec): {X_doc_train.shape}")
print(f"Shape of y_train (doc2vec): {y_doc_train.shape}")
print(f"Shape of X_test (dov2vec): {X_doc_test.shape}")
print(f"Shape of y_test (doc2vec): {y_doc_test.shape}")



Shape of X_train: (6430, 22)
Shape of y_train: (6430, 1)
Shape of X_test: (2757, 22)
Shape of y_test: (2757, 1)
Shape of X_train (minimal): (6430, 4)
Shape of y_train (minimal): (6430, 1)
Shape of X_test (minimal): (2757, 4)
Shape of y_test (minimal): (2757, 1)
Shape of X_train (word2vec): (4606, 124)
Shape of y_train (word2vec): (4606, 1)
Shape of X_test (word2vec): (1975, 124)
Shape of y_test (word2vec): (1975, 1)
Shape of X_train (doc2vec): (4608, 123)
Shape of y_train (doc2vec): (4608, 1)
Shape of X_test (dov2vec): (1975, 123)
Shape of y_test (doc2vec): (1975, 1)


In [106]:
# classifiers
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


#Decision Tree classifier
def runDecisionTree(x_train, y_train, x_test, y_test):
    classifier = tree.DecisionTreeClassifier()
    #parameters for decision tree
    #parameters = what the model learns
    #hyperparameters = hopw the model learns

    #copied from github
    #parameter grid

    parameter_grid = {'criterion': ['gini', 'entropy'],
                  'splitter': ['best', 'random'],
                  'min_samples_split': [3, 4], # reg
                  'max_depth': [1, 2, 3, 4], # regularize
                  'min_samples_leaf': [2,3], # reg
                  'max_features': list(range(1,len(x_train.columns)+1))
                      }

    #grid search for finding hyperparameters
    grid_search = GridSearchCV(classifier, param_grid=parameter_grid, return_train_score=True,  n_jobs=20, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train, y_train)

    #output the best scores and the best performing parameters
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    #create best features and training data
    classifier = grid_search.best_estimator_

    #make the predictions
    decisiontree_predict = classifier.predict(x_test)

    #accuracy on the test data
    decisiontree_score = accuracy_score(y_test, decisiontree_predict)
    print("Decision Tree Accuracy on the testing data is " + str(decisiontree_score), file= sys.stderr)

    """
    # let's compare it to a naive algorithm that
    # simply selects the order code at random
    random_sample = np.random.choice([0,1], len(y_test), replace=True)
    r_score = accuracy_score(y_test, random_sample)
    print("random algorithm classification accuracy on test data is " + str(r_score), file=sys.stderr)
    """

    #print feature importances; the most influential features in the dataset
    print("feature importances " + str(classifier.feature_importances_), file=sys.stderr)

    #ranking of the features importantness
    # Print the feature ranking
    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names':[],'imp':[]}
    for name, importance in zip(x_train.columns, classifier.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp',ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f"%(i, row['names'],row['imp']), file=sys.stderr)
        i += 1
    
    #extract results from grid search
    crossval_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = crossval_results.sort_values(by="rank_test_score").head()

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)
    
    #this might be redundant
    decisiontree_predict = classifier.predict(x_test)
    decisiontree_score = accuracy_score(y_test,decisiontree_predict)

    
    print("accuracy score on test data: " + str(decisiontree_score), file=sys.stderr)
    train_score = accuracy_score(y_train, classifier.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)

    return train_score, decisiontree_score

#run on minimal dataset

"""
X_min_train, X_min_test, y_min_train, y_min_test
"""
#minimal
print("Minimal scores:")    
runDecisionTree(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runDecisionTree(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Merged Scores:")
runDecisionTree(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print("doc2vec merged scores:")

runDecisionTree(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)



    



Minimal scores:


Best score: 0.5524105754276827
Best parameters: {'criterion': 'gini', 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 4, 'splitter': 'best'}
Decision Tree Accuracy on the testing data is 0.5317373957199855
feature importances [0.41740018 0.21288177 0.23303351 0.13668454]
Feature ranking:
0. CaseReferenceNumber 0.417400
1. CaseAttorneyJuris 0.233034
2. AttorneySpecialization 0.212882
3. MotionTimeDuration 0.136685

Top 5 best Parameters: 
1. {'criterion': 'gini', 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 4, 'splitter': 'best'} train: 0.5590634180058753 test: 0.5524105754276827
2. {'criterion': 'entropy', 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 3, 'splitter': 'best'} train: 0.5562294798686712 test: 0.5471228615863142
3. {'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 4, 'splitter': 'best'} train: 0.5517020908933816 test: 0.5469

Subset Scores


Best score: 0.5892690513219285
Best parameters: {'criterion': 'gini', 'max_depth': 4, 'max_features': 17, 'min_samples_leaf': 2, 'min_samples_split': 3, 'splitter': 'best'}
Decision Tree Accuracy on the testing data is 0.5553137468262604
feature importances [0.07236214 0.13782653 0.02938287 0.05205616 0.         0.44135492
 0.19859989 0.         0.         0.04631362 0.         0.
 0.         0.         0.         0.         0.         0.02210386
 0.         0.         0.         0.        ]
Feature ranking:
0. CaseMajorCode_V 0.441355
1. MotionTimeDuration 0.198600
2. AttorneySpecialization 0.137827
3. CaseReferenceNumber 0.072362
4. MotionJurisNumber 0.052056
5. CaseLocation_FBT 0.046314
6. MotionJurisNumber 0.029383
7. CaseLocation_NNH 0.022104
8. CaseLocation_KNO 0.000000
9. CaseLocation_UWY 0.000000
10. CaseLocation_TTD 0.000000
11. CaseLocation_NNI 0.000000
12. CaseLocation_MMX 0.000000
13. CaseLocation_LLI 0.000000
14. CaseLocation_HHB 0.000000
15. CaseLocation_KNL 0.000000
16. 

Merged Scores:
doc2vec merged scores:


Best score: 0.5801188342921815
Best parameters: {'criterion': 'entropy', 'max_depth': 4, 'max_features': 14, 'min_samples_leaf': 3, 'min_samples_split': 3, 'splitter': 'random'}
Decision Tree Accuracy on the testing data is 0.5524050632911393
feature importances [0.         0.         0.         0.01749544 0.54419451 0.
 0.         0.         0.         0.         0.05489635 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.01711312 0.         0.
 0.01701454 0.         0.         0.         0.         0.
 0.         0.02127552 0.         0.         0.01659585 0.
 0.         0.00860323 0.         0.0411251  0.         0.
 0.         0.         0.         0.08836602 0.         0.
 0.         0.         0.         0.02530551 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.    

ValueError: Found input variables with inconsistent numbers of samples: [4608, 1975]

In [111]:
print("doc2vec merged scores:")

runDecisionTree(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)


doc2vec merged scores:


Best score: 0.5909237951523154
Best parameters: {'criterion': 'entropy', 'max_depth': 4, 'max_features': 78, 'min_samples_leaf': 3, 'min_samples_split': 4, 'splitter': 'best'}
Decision Tree Accuracy on the testing data is 0.5711392405063291
feature importances [0.         0.05728171 0.         0.         0.22665449 0.
 0.13384281 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.06006514
 0.         0.         0.03827092 0.04322819 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.03839394 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         

(0.619140625, 0.5711392405063291)

In [115]:
from sklearn.ensemble import GradientBoostingClassifier 


def runGradientBoostingClassifier(x_train,y_train,x_test,y_test):

    # Here we instantiate the gradient boosting classifier
    clf = GradientBoostingClassifier()

    parameter_grid = {
        "learning_rate": np.linspace(0.001, .01, 3),# best below .1
        "min_samples_split": np.linspace(.3, .5, 2),  # original bot .1 to .5 with 12 - reg
        "min_samples_leaf": np.linspace(.3, .5, 2), # reg
        'max_depth': [1, 2, 3], # reg
        "subsample":[0.5, 0.6, 0.7], # reg
        "n_estimators":[100, 200, 300 ] # reg - larger slower to overfit
    }

    grid_search = GridSearchCV(clf, n_jobs=40, return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    # refit and train the model to the best features and training data
    clf = grid_search.best_estimator_

    # now we have to computer the classification accuracy
    # think about what two variables we have to compare
    gbc_predictions = clf.predict(x_test)
    dt_score = accuracy_score(y_test, gbc_predictions)
    print("accuracy score on test data: " +str(dt_score), file=sys.stderr)
    print("feature importances " + str(clf.feature_importances_), file=sys.stderr)

    # Print the feature ranking
    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names':[],'imp':[]}
    for name, importance in zip(x_train.columns, clf.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp', ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f"%(i, row['names'],row['imp']),file=sys.stderr)
        i += 1

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)



    etc_predictions = clf.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)

    return (train_score,dt_score)



#minimal
print("Minimal scores:")    
runGradientBoostingClassifier(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runGradientBoostingClassifier(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
runGradientBoostingClassifier(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
runGradientBoostingClassifier(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)





Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.53950233281493
Best parameters: {'learning_rate': 0.001, 'max_depth': 1, 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100, 'subsample': 0.5}
accuracy score on test data: 0.539354370692782
feature importances [0. 0. 0. 0.]
Feature ranking:
0. CaseReferenceNumber 0.000000
1. AttorneySpecialization 0.000000
2. CaseAttorneyJuris 0.000000
3. MotionTimeDuration 0.000000

Top 5 best Parameters: 
1. {'learning_rate': 0.001, 'max_depth': 1, 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100, 'subsample': 0.5} train: 0.5395023328149302 test: 0.53950233281493
1. {'learning_rate': 0.0

Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.5639191290824261
Best parameters: {'learning_rate': 0.01, 'max_depth': 1, 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 300, 'subsample': 0.7}
accuracy score on test data: 0.5636561479869423
feature importances [0.0266221  0.10569811 0.04048723 0.04696209 0.27965275 0.33399612
 0.16658161 0.         0.         0.         0.         0.
 0

Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.584895312647364
Best parameters: {'learning_rate': 0.01, 'max_depth': 2, 'min_samples_leaf': 0.3, 'min_samples_split': 0.5, 'n_estimators': 300, 'subsample': 0.7}
accuracy score on test data: 0.5660759493670886
feature importances [0.00605444 0.         0.00611036 0.01204252 0.25262962 0.24148232
 0.12959012 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.12482844 0.         0.         0.00609096
 0.         0.         0.         0.         0.         0.
 0.         0.      

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.5913684806186927
Best parameters: {'learning_rate': 0.01, 'max_depth': 1, 'min_samples_leaf': 0.3, 'min_samples_split': 0.5, 'n_estimators': 300, 'subsample': 0.7}
accuracy score on test data: 0.5888607594936709
feature importances [0.         0.         0.         0.         0.11893534 0.14364242
 0.05290051 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00572189 0.         0.0017423
 0.01139856 0.         0.         0.01739892 0.01083274 0.
 0.         0.093623

(0.6009114583333334, 0.5888607594936709)

In [114]:
def runAdaBoostClassifier(x_train,y_train,x_test,y_test):

    # Here we instantiate the adaboost classifier
    clf = AdaBoostClassifier()

    parameter_grid = {
    "estimator": [tree.DecisionTreeClassifier(splitter='random',max_depth=1)], # lower depth and min sample split
    "learning_rate": np.linspace(0.0001, .001, 10),
    "n_estimators":[100,300,500]
    }

    grid_search = GridSearchCV(clf, n_jobs=20,  return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    # refit and train the model to the best features and training data
    clf = grid_search.best_estimator_

    print("feature importances " + str(clf.feature_importances_), file=sys.stderr)

    # Print the feature ranking
    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names':[],'imp':[]}
    for name, importance in zip(x_train.columns, clf.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp',ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f"%(i, row['names'],row['imp']),file=sys.stderr)
        i += 1

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)

    # now we have to computer the classification accuracy
    # think about what two variables we have to compare
    etc_predictions = clf.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)
    return (train_score,dt_score)




print("Minimal scores:")    
runAdaBoostClassifier(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runAdaBoostClassifier(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
runAdaBoostClassifier(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
runAdaBoostClassifier(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)



Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Best score: 0.5399688958009332
Best parameters: {'estimator': DecisionTreeClassifier(max_depth=1, splitter='random'), 'learning_rate': 0.00019999999999999998, 'n_estimators': 300}
feature importances [0.57666667 0.14333333 0.24333333 0.03666667]
Feature ranking:
0. CaseReferenceNumber 0.576667


Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Best score: 0.5741835147744945
Best parameters: {'estimator': DecisionTreeClassifier(max_depth=1, splitter='random'), 'learning_rate': 0.0001, 'n_estimators': 100}
feature importances [0.   0.   0.   0.   0.43 0.57 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.  ]


Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Best score: 0.5729510515891729
Best parameters: {'estimator': DecisionTreeClassifier(max_depth=1, splitter='random'), 'learning_rate': 0.0001, 'n_estimators': 100}
feature importances [0.   0.   0.   0.   0.54 0.46 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Best score: 0.5722705837970385
Best parameters: {'estimator': DecisionTreeClassifier(max_depth=1, splitter='random'), 'learning_rate': 0.0001, 'n_estimators': 100}
feature importances [0.   0.   0.   0.   0.51 0.49 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0

(0.572265625, 0.5589873417721519)

In [116]:
def runExtraTreesClassifier(x_train,y_train,x_test,y_test):

    # Here we instantiate the extra tree classifier
    clf = ExtraTreesClassifier()

    parameter_grid = {
        "criterion": ["gini",  "entropy"],
        "n_estimators":[100,150,200,250,300,400,500],
        "max_depth":[2,3,4], # reg
        "min_samples_split": np.linspace(.3, .5, 5), # original bot .1 to .5 with 12
        "min_samples_leaf": np.linspace(.3, .5, 5),
        "max_features": ["log2", "sqrt"]
    }

    grid_search = GridSearchCV(clf, n_jobs=20,  return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    # refit and train the model to the best features and training data
    clf = grid_search.best_estimator_

    importances = clf.feature_importances_
    print(importances, file=sys.stderr)
    # Print the feature ranking
    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names':[],'imp':[]}
    for name, importance in zip(x_train.columns, clf.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp',ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f"%(i, row['names'],row['imp']),file=sys.stderr)
        i += 1

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)

    # now we have to computer the classification accuracy
    # think about what two variables we have to compare
    etc_predictions = clf.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)

    return (train_score,dt_score)



print("Minimal scores:")    
runExtraTreesClassifier(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runExtraTreesClassifier(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
runExtraTreesClassifier(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
runExtraTreesClassifier(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)


Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.53950233281493
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100}
[0.73684211 0.15789474 0.02631579 0.07894737]
Feature ranking:
0. CaseReferenceNumber 0.736842
1. AttorneySpecialization 0.157895
2. MotionTimeDuration 0.078947
3. CaseAttorneyJuris 0.026316

Top 5 best Parameters: 
1. {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100} train: 0.5395023328149302 test: 0.53950233281493
1. {'criterion': 'entropy', 'max_depth': 3, 'max_featu

Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.5430793157076205
Best parameters: {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.3, 'min_samples_split': 0.5, 'n_estimators': 100}
[0.29787234 0.         0.         0.         0.34042553 0.34042553
 0.0212766  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
Feature ranking:
0. CaseMajorCode_T 0.340426
1. CaseMajorCode_V 0.340426
2. CaseReferenceNumber 0.297872
3. MotionTimeDuration 0.021277
4. CaseLocation_KNL 0.000000
5. CaseLocation_UWY 0.000000
6. CaseLocation_TTD 0.0000

Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.5312633217014053
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100}
[0.         0.         0.         0.         0.11111111 0.03703704
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.07407407 0.07407407
 0.         0.07407407 0.03703704 0.         0.         0.
 0.03703704 0.         0.         0.         0.         0.03703704
 0.03703704 0.         0.         0.         0.         0.
 0.         0.   

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.5312496463265114
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100}
[0.         0.         0.         0.         0.13040138 0.10526316
 0.02631579 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.07894737 0.
 0.         0.02631579 0.         0.         0.         0.02631579
 0.         0.         0.         0.         0.         0.
 0.         0.02631579 0.         0.         0.         0.
 0.         0.         0.

(0.53125, 0.5311392405063291)

In [117]:
def runSupportVec(x_train,y_train,x_test,y_test):

    # Here we instantiate the support vector classifier
    clf = svm.SVC()
    parameter_grid = {'C': np.linspace(0.001,.1,100),
                        'gamma': [1, 0.1, 0.001, 0.0001],
                        'kernel': ['rbf']
                      }

    grid_search = GridSearchCV(clf, n_jobs=40, return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    # refit and train the model to the best features and training data
    clf = grid_search.best_estimator_

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)

    # now we have to computer the classification accuracy
    # think about what two variables we have to compare
    etc_predictions = clf.predict(x_test)
    dt_score = accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)
    
    return (train_score, dt_score)




print("Minimal scores:")    
runSupportVec(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runSupportVec(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
runSupportVec(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
runSupportVec(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)


Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.53950233281493
Best parameters: {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'}

Top 5 best Parameters: 
1. {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.069, 'gamma': 1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.0001, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.001, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
accuracy score on test data: 0.5393543706927

Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.53950233281493
Best parameters: {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'}

Top 5 best Parameters: 
1. {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.069, 'gamma': 1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.0001, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.001, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
1. {'C': 0.068, 'gamma': 0.1, 'kernel': 'rbf'} train: 0.5395023328149302 test: 0.53950233281493
accuracy score on test data: 0.5393543706927

Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.5312633217014053
Best parameters: {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'}

Top 5 best Parameters: 
1. {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'} train: 0.5312635662027201 test: 0.5312633217014053
1. {'C': 0.069, 'gamma': 1, 'kernel': 'rbf'} train: 0.5312635662027201 test: 0.5312633217014053
1. {'C': 0.068, 'gamma': 0.0001, 'kernel': 'rbf'} train: 0.5312635662027201 test: 0.5312633217014053
1. {'C': 0.068, 'gamma': 0.001, 'kernel': 'rbf'} train: 0.5312635662027201 test: 0.5312633217014053
1. {'C': 0.068, 'gamma': 0.1, 'kernel': 'rbf'} train: 0.5312635662027201 test: 0.5312633217014053
accuracy score on test data: 0.5

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Best score: 0.5312496463265114
Best parameters: {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'}

Top 5 best Parameters: 
1. {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'} train: 0.53124999563998 test: 0.5312496463265114
1. {'C': 0.069, 'gamma': 1, 'kernel': 'rbf'} train: 0.53124999563998 test: 0.5312496463265114
1. {'C': 0.068, 'gamma': 0.0001, 'kernel': 'rbf'} train: 0.53124999563998 test: 0.5312496463265114
1. {'C': 0.068, 'gamma': 0.001, 'kernel': 'rbf'} train: 0.53124999563998 test: 0.5312496463265114
1. {'C': 0.068, 'gamma': 0.1, 'kernel': 'rbf'} train: 0.53124999563998 test: 0.5312496463265114
accuracy score on test data: 0.53113924050

(0.53125, 0.5311392405063291)

In [128]:
from xgboost.sklearn import XGBClassifier
def runXGBoost(x_train,y_train,x_test,y_test):
    # Store column names before converting to numpy
    if isinstance(x_train, pd.DataFrame):
        feature_names = x_train.columns
        x_train = x_train.to_numpy()
    if isinstance(x_test, pd.DataFrame):
        x_test = x_test.to_numpy()
    if isinstance(y_train, pd.Series):
        y_train = y_train.to_numpy()
    if isinstance(y_test, pd.Series):
        y_test = y_test.to_numpy()
        
    parameter_grid = {
        'reg_lambda': np.linspace(27, 28 , 2),  # for over fitting
        'reg_alpha': np.linspace(27, 28, 2),
        "learning_rate": np.linspace(.0001, .001, 3), # usually between .05 and .3
        "max_depth": [2],
        "num_boosting_rounds": [1000],
        'nthread': [10]
    }

    clf = XGBClassifier()
    grid_search = GridSearchCV(clf, n_jobs=40, return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)

    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    clf = grid_search.best_estimator_
    importances = clf.feature_importances_
    print(importances, file=sys.stderr)

    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names': [], 'imp': []}
    # Use stored feature_names instead of x_train.columns
    for name, importance in zip(feature_names, clf.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
        
    # Rest of your code remains the same
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp', ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f" % (i, row['names'], row['imp']), file=sys.stderr)
        i += 1

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)

    etc_predictions = clf.predict(x_test)
    dt_score = accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)

    return (train_score, dt_score)


print("Minimal scores:")    
runXGBoost(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
runXGBoost(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
runXGBoost(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
runXGBoost(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)


Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:14:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "num_boosting_rounds" } are not used.

  warnings.warn(smsg, UserWarning)
Best score: 0.53950233281493
Best parameters: {'learning_rate': 0.0001, 'max_depth': 2, 'nthread': 10, 'num_boosting_rounds': 1000, 'reg_alpha': 27.0, 'reg_lambda': 27.0}
[0.         0.43490005 0.         0.56509995]
Feature ranking:
0. MotionTimeDuration 0.565100
1. AttorneySpecialization 0.434900
2. CaseReferenceNumber 0.000000
3. CaseAttorneyJuris 0.000000

Top 5 best Parameters: 
1. {'learning_rate': 0.0001, 'max_depth': 2, 'nthread': 10, 'num_boosting_rounds': 1000, 'reg_alpha': 27.0, 'reg_lambda': 27.0} train: 0.5395023328149302 test: 0.53950233281493
1. {'learning_rate'

Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:14:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "num_boosting_rounds" } are not used.

  warnings.warn(smsg, UserWarning)
Best score: 0.53950233281493
Best parameters: {'learning_rate': 0.0001, 'max_depth': 2, 'nthread': 10, 'num_boosting_rounds': 1000, 'reg_alpha': 27.0, 'reg_lambda': 27.0}
[0.         0.         0.         0.         0.8638487  0.
 0.13615133 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
Feature ranking:
0. CaseMajorCode_T 0.863849
1. MotionTimeDuration 0.136151
2. CaseReferenceNumber 0.000000
3. CaseLocation_HHD 0.000000
4. CaseLocation_UWY 0.000000
5. CaseLocation_TTD 0.0

Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:14:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "num_boosting_rounds" } are not used.

  warnings.warn(smsg, UserWarning)
Best score: 0.5312633217014053
Best parameters: {'learning_rate': 0.0001, 'max_depth': 2, 'nthread': 10, 'num_boosting_rounds': 1000, 'reg_alpha': 27.0, 'reg_lambda': 27.0}
[0.         0.         0.         0.         0.73459893 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:15:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "num_boosting_rounds" } are not used.

  warnings.warn(smsg, UserWarning)
Best score: 0.5312496463265114
Best parameters: {'learning_rate': 0.0001, 'max_depth': 2, 'nthread': 10, 'num_boosting_rounds': 1000, 'reg_alpha': 27.0, 'reg_lambda': 27.0}
[0.         0.         0.         0.         0.58684236 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

(0.53125, 0.5311392405063291)

In [119]:
!pip install xgboost scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
    --------------------------------------- 1.6/124.9 MB 12.0 MB/s eta 0:00:11
   -- ------------------------------------- 8.4/124.9 MB 26.0 MB/s eta 0:00:05
   ---- ----------------------------------- 14.9/124.9 MB 28.4 MB/s eta 0:00:04
   ------- -------------------------------- 24.4/124.9 MB 32.9 MB/s eta 0:00:04
   ---------- ----------------------------- 33.0/124.9 MB 35.0 MB/s eta 0:00:03
   ------------- -------------------------- 41.7/124.9 MB 35.8 MB/s eta 0:00:03
   --------------- ------------------------ 49.0/124.9 MB 35.9 MB/s eta 0:00:03
   ----------------- ---------------------- 55.6/124.9 MB 35.4 MB/s eta 0:00:02
   -------------------- ------------------- 64.2/124.9 MB 35.9 MB/s eta 0:00:02
   ----------------------- ---------------- 72.1/124.9 MB 36.2 MB/s eta 0:00:02
   ------------------------- -------------- 80.5/124.

In [122]:
def random_trees(x_train,y_train,x_test,y_test):

    # Here we instantiate the random forest classifier
    clf = RandomForestClassifier()
    parameter_grid = {"n_estimators": [100, 200, 250, 300, 400, 500],
                  "criterion": ["gini",  "entropy"],
                  "max_depth":[2,3,4,5],
                  "max_features": ["log2", "sqrt"],
                  "min_samples_split": np.linspace(0.3, 0.5, 10),
                  "min_samples_leaf": np.linspace(0.3, 0.5, 10),
                  'n_jobs': [10]
    }

    grid_search = GridSearchCV(clf, n_jobs=20,  return_train_score=True, param_grid=parameter_grid, cv=StratifiedKFold(n_splits=10))
    grid_search.fit(x_train,y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    # refit and train the model to the best features and training data
    clf = grid_search.best_estimator_

    importances = clf.feature_importances_
    print(importances, file=sys.stderr)
    # Print the feature ranking
    print("Feature ranking:", file=sys.stderr)
    importanceDict = {'names': [], 'imp': []}
    for name, importance in zip(x_train.columns, clf.feature_importances_):
        importanceDict['names'] += [name]
        importanceDict['imp'] += [importance]
    fRank = pd.DataFrame.from_dict(importanceDict)
    fRank = fRank.sort_values(by='imp', ascending=False)
    i = 0
    for index, row in fRank.iterrows():
        print("%d. %s %f" % (i, row['names'], row['imp']), file=sys.stderr)
        i += 1

    cv_results = pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'params','mean_test_score','mean_train_score']]
    sorted_results = cv_results.sort_values(by='rank_test_score').head(5)

    print("\nTop 5 best Parameters: ", file=sys.stderr)
    for index, row in sorted_results.iterrows():
        print("%d. %s train: %s test: %s" % (row['rank_test_score'], str(row['params']), str(row['mean_train_score']), str(row['mean_test_score'])), file=sys.stderr)

    # now we have to computer the classification accuracy
    # think about what two variables we have to compare
    etc_predictions = clf.predict(x_test)
    dt_score = accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, clf.predict(x_train))
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)

    return (train_score, dt_score)





print("Minimal scores:")    
random_trees(X_min_train,y_min_train,X_min_test,y_min_test)

#subset
print("Subset Scores")
random_trees(X_train,y_train,X_test,y_test)

#combined Word2Vec
print("Word2Vec Combined Scores")
random_trees(X_merge_train,y_merge_train,X_merge_test,y_merge_test)

#combined Doc2Vec
print('Doc2vec Scores')
random_trees(X_doc_train, y_doc_train ,X_doc_test , y_doc_test)


Minimal scores:


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.53950233281493
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100, 'n_jobs': 10}
[0.06 0.42 0.29 0.23]
Feature ranking:
0. AttorneySpecialization 0.420000
1. CaseAttorneyJuris 0.290000
2. MotionTimeDuration 0.230000
3. CaseReferenceNumber 0.060000

Top 5 best Parameters: 
1. {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100, 'n_jobs': 10} train: 0.5395023328149302 test: 0.53950233281493
1. {'criterion': 'entropy', 'max_depth': 3, 'max_f

Subset Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.53950233281493
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3, 'n_estimators': 100, 'n_jobs': 10}
[0.10294118 0.26470588 0.17647059 0.23529412 0.         0.
 0.22058824 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
Feature ranking:
0. AttorneySpecialization 0.264706
1. MotionJurisNumber 0.235294
2. MotionTimeDuration 0.220588
3. MotionJurisNumber 0.176471
4. CaseReferenceNumber 0.102941
5. CaseLocation_KNL 0.000000
6. CaseLocation

Word2Vec Combined Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.5347363953598039
Best parameters: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.3, 'min_samples_split': 0.3222222222222222, 'n_estimators': 100, 'n_jobs': 10}
[0.01 0.   0.   0.   0.   0.   0.03 0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.07 0.11 0.
 0.   0.07 0.   0.   0.   0.03 0.   0.01 0.01 0.01 0.01 0.   0.   0.
 0.   0.   0.03 0.   0.   0.   0.01 0.   0.   0.01 0.01 0.   0.   0.01
 0.   0.   0.01 0.   0.   0.01 0.   0.   0.01 0.01 0.04 0.   0.   0.02
 0.02 0.   0.04 0.   0.03 0.   0.   0.   0.01 0.   0.   0.   0.   0.
 0.   0

Doc2vec Scores


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Best score: 0.553604640196171
Best parameters: {'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 0.3, 'min_samples_split': 0.34444444444444444, 'n_estimators': 100, 'n_jobs': 10}
[0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.02
 0.   0.01 0.   0.   0.   0.   0.04 0.   0.   0.04 0.13 0.   0.02 0.02
 0.   0.   0.   0.01 0.01 0.04 0.   0.   0.   0.   0.   0.09 0.   0.02
 0.   0.   0.   0.01 0.   0.   0.   0.13 0.03 0.   0.02 0.01 0.   0.
 0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.
 0.  

(0.5609809027777778, 0.5488607594936709)